In [65]:
import pandas as pd

In [123]:
df_22 = pd.read_csv ('../data/NagsHead202212.CSV', header= None, names = ['SP', 'X', 'Y', 'Z'])
df_23 = pd.read_csv ('../data/NagsHead202303.CSV', header= None, names = ['SP', 'X', 'Y', 'Z'])

In [136]:
sites = pd.read_csv ('../data/Sites.csv')
sites

,Street name,Lat,Lon
0,E Hollowell St,35.968790,-75.629492
1,Grouse St,35.917644,-75.600467
2,E Hardgrove St,35.898667,-75.590467
3,E Tides Dr,35.891257,-75.585988
4,E June St,35.875947,-75.578200
5,Juncos St,35.869530,-75.574821
6,E Pelican St,35.859092,-75.569298
7,E Proteus Ct,35.858081,-75.568677
8,E Seagull Dr,35.854635,-75.567069
9,10333A S Old Oregon Inlet Rd,35.850184,-75.565215


In [149]:
import utm
res = pd.DataFrame()
for index, row in sites.iterrows(): 
        e, n, zone, letter = utm.from_latlon(row['Lat'], row['Lon']) # utm.from_latlon(LATITUDE, LONGITUDE).
        #res.at[index, 'Name'] = row['Street name']
        res.at[index, 'E'] = e
        res.at[index, 'N'] = n

In [150]:
res.to_csv('../data/sites_utm.csv', index=False , header=None)

In [125]:
import math

def ecef_to_geodetic(x, y, z):
    # WGS84 ellipsoid parameters
    a = 6378137.0  # semi-major axis in meters
    f_inv = 298.257223563  # inverse flattening

    # Calculate eccentricity squared (e^2)
    e2 = 1.0 - (1.0 - 1.0 / f_inv) ** 2

    # Calculate longitude
    lon = math.atan2(y, x)

    # Calculate distance from the origin
    p = math.sqrt(x**2 + y**2)

    # Calculate latitude using iterative approach
    lat = math.atan2(z, p * (1.0 - e2))

    # Iterate to refine latitude using the iterative method
    while True:
        N = a / math.sqrt(1.0 - e2 * math.sin(lat)**2)
        new_lat = math.atan2(z + e2 * N * math.sin(lat), p)
        if abs(lat - new_lat) < 1e-10:
            break
        lat = new_lat

    # Calculate altitude
    alt = p / math.cos(lat) - N

    # Convert latitude and longitude to degrees
    lat = math.degrees(lat)
    lon = math.degrees(lon)

    return lat, lon, alt

# Example usage
ecef_x = 1290218.338     # replace with your ECEF X coordinate
ecef_y = -5012337.169  # replace with your ECEF Y coordinate
ecef_z = 3714704.488  # replace with your ECEF Z coordinate

latitude, longitude, altitude = ecef_to_geodetic(ecef_x, ecef_y, ecef_z)

print(f"Latitude: {latitude} degrees")
print(f"Longitude: {longitude} degrees")
print(f"Altitude: {altitude} meters")


Latitude: 35.85010770362138 degrees
Longitude: -75.56494998904823 degrees
Altitude: -32.60475525725633 meters


In [126]:
# Loop through every row using iterrows()
for index, row in df_22.iterrows(): 
    latitude, longitude, altitude = ecef_to_geodetic(row['X'], row['Y'], row['Z'])
    df_22.loc[index, 'lat'] = latitude
    df_22.loc[index, 'lon'] = longitude
    df_22.loc[index, 'alt'] = altitude

In [127]:
for index, row in df_23.iterrows(): 
    latitude, longitude, altitude = ecef_to_geodetic(row['X'], row['Y'], row['Z'])
    df_23.loc[index, 'lat'] = latitude
    df_23.loc[index, 'lon'] = longitude
    df_23.loc[index, 'alt'] = altitude

In [177]:
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import folium

m = folium.Map(location=[35.85018, -75.565215], zoom_start=18 ,width=600, height= 500)
folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(m)
for index, row in df_22.iterrows(): 
        radius = 5
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=radius,
            color="darkblue",
            stroke=False,
            fill=True,
            fill_opacity=1,
            opacity=1,
            popup="{} ".format(row['SP']),
            tooltip="{} ".format(2022),
        ).add_to(m)
for index, row in df_23.iterrows(): 
        radius =  5
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=radius,
            color="red",
            stroke=False,
            fill=True,
            fill_opacity=1,
            opacity=1,
            popup="{} ".format(row['SP']),
            tooltip="{} ".format(2023),
        ).add_to(m)
for index, row in sites.iterrows():  # Street name	Lat	Lon
        radius =  10
        folium.CircleMarker(
            location=[row['Lat'], row['Lon']],
            radius=radius,
            color="yellow",
            stroke=False,
            fill=True,
            fill_opacity=1,
            opacity=1,
            popup="{} ".format(row['Street name']),
            tooltip="{} ".format(row['Street name']),
        ).add_to(m)
m

In [176]:
200/60*2.5

8.333333333333334